In [1]:
from carbon import CarbonSimulatorUI
from carbon.simulators.carbon_simulator import (
    __version__ as uiversion,
    __date__ as uidate,
)
from jupyformat import *

In [2]:
print("[carbon_simulator] version", uiversion, uidate)
jp()

[carbon_simulator] version 0.9.10.1 12/Nov/2022


# Carbon Simulation - Test 24

In [3]:
Sim = CarbonSimulatorUI(pair="ETHUSDC", raiseonerror=True, verbose=False)
Sim

CarbonSimulatorUI(<0 positions, 0 trades>, pair='ETHUSDC')

In [4]:
help(Sim.add_linked_pos)

Help on method add_linked_pos in module carbon.simulators.carbon_simulator:

add_linked_pos(tkn: str, amt: Any, psell1: Any, psell2: Any, amtr: Any, pbuy1: Any, pbuy2: Any, pair: str = None) -> Dict[str, Any] method of carbon.simulators.carbon_simulator.CarbonSimulatorUI instance
    adds two linked position (one buy, one sell)
    
    :tkn:       the token that is sold in the range psell1/2, eg "ETH"*
    :amt:       the amount of `tkn` that is available for sale in range psell1/2
    :psell1/2:  both ends of the sell `tkn` range**, quoted in the price convention of `pair`
    :amtr:      the amount of the other token that is available for selling against tkn in range pbuy1/2
    :pbuy1/2:   both ends of the buy `tkn` range**, quoted in the price convention of `pair`
    :pair:      the token pair to which the position corresponds, eg "ETHUSD"*
    
    *if the pair is not available in the simulator, or if `tkn` is not part of `pair`
    an exception is thrown or an error record is r

We assert that we can not add an unrelated position to this pair, either using an existing token like LINK, or a token like DNE (does not exist).

In [5]:
Sim.add_linked_pos("ETH", 10, 2000, 2000, 10000, 1000, 1000)["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,10.000000,10.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,10000.000000,10000.000000,USDC,1000.000000,1000.000000,USDC per ETH,0


In [6]:
ji("below we are checking that (a) buying and selling is at the correct price [2000 for selling ETH and buying USDC, 1000 for buying ETH and selling USDC] and (b) that the `y` variables on the respective curves adjust, and possibly the `y_int` if curves need to be expanded")

## Sell ETH

In [7]:
Sim.amm_sells("ETH", 1)["trades"]

,uid,id,sub_id,note,aggr,amt1,tkn1,amt2,tkn2,pair,route_indexes,num_trades,price_avg,p_unit
0,0.0,0,0,route #0,False,1.0,ETH,2000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,0,0,None,AMM sells 1ETH buys 2000USDC,True,1.0,ETH,2000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


In [8]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,10.000000,9.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,12000.000000,12000.000000,USDC,1000.000000,1000.000000,USDC per ETH,0


## Buy ETH

In [9]:
Sim.amm_buys("ETH", 1)["trades"]

,uid,id,sub_id,note,aggr,amt1,tkn1,amt2,tkn2,pair,route_indexes,num_trades,price_avg,p_unit
0,1.0,1,0,route #1,False,1000.0,USDC,1.0,ETH,ETHUSDC,1,1,1000.0,USDC per ETH
0,1,1,None,AMM sells 1000USDC buys 1ETH,True,1000.0,USDC,1.0,ETH,ETHUSDC,[1],1,1000.0,USDC per ETH


In [10]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,10.000000,10.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,12000.000000,11000.000000,USDC,1000.000000,1000.000000,USDC per ETH,0


## Sell USDC

In [11]:
Sim.amm_sells("USDC", 2000)["trades"]

,uid,id,sub_id,note,aggr,amt1,tkn1,amt2,tkn2,pair,route_indexes,num_trades,price_avg,p_unit
0,2.0,2,0,route #1,False,2000.0,USDC,2.0,ETH,ETHUSDC,1,1,1000.0,USDC per ETH
0,2,2,None,AMM sells 2000USDC buys 2ETH,True,2000.0,USDC,2.0,ETH,ETHUSDC,[1],1,1000.0,USDC per ETH


In [12]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,12.000000,12.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,12000.000000,9000.000000,USDC,1000.000000,1000.000000,USDC per ETH,0


## Buy USDC

In [13]:
Sim.amm_buys("USDC", 2000)["trades"]

,uid,id,sub_id,note,aggr,amt1,tkn1,amt2,tkn2,pair,route_indexes,num_trades,price_avg,p_unit
0,3.0,3,0,route #0,False,1.0,ETH,2000.0,USDC,ETHUSDC,0,1,2000.0,USDC per ETH
0,3,3,None,AMM sells 1ETH buys 2000USDC,True,1.0,ETH,2000.0,USDC,ETHUSDC,[0],1,2000.0,USDC per ETH


In [14]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,12.000000,11.000000,ETH,2000.000000,2000.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,12000.000000,11000.000000,USDC,1000.000000,1000.000000,USDC per ETH,0
